In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

data_dir = "/content/drive/MyDrive/music-project-ml-tokenized"
print(os.listdir(data_dir))


['vocab.json', 'train_ids.npy', 'test_ids.npy', 'val_ids.npy']


In [3]:
combined_ckpt_path = "/content/drive/MyDrive/music-project-ml-results/best_xl_full_checkpoint.pt"

In [4]:
import torch
import torch.nn as nn
import math

class GPTConfig:
    def __init__(self,
                 vocab_size: int,
                 block_size: int,
                 n_layer: int,
                 n_head: int,
                 n_embd: int,
                 dropout: float = 0.1):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout


class CausalSelfAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.head_dim = config.n_embd // config.n_head

        self.qkv = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        self.attn_drop = nn.Dropout(config.dropout)
        self.resid_drop = nn.Dropout(config.dropout)

        self.register_buffer(
            "mask",
            torch.tril(torch.ones(config.block_size, config.block_size))
            .view(1, 1, config.block_size, config.block_size)
        )

    def forward(self, x):
        B, T, C = x.size()

        qkv = self.qkv(x)  # (B, T, 3*C)
        q, k, v = qkv.split(C, dim=2)

        q = q.view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_head, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B, nh, T, T)
        att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float("-inf"))
        att = torch.softmax(att, dim=-1)
        att = self.attn_drop(att)

        y = att @ v  # (B, nh, T, hd)
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        y = self.resid_drop(self.proj(y))
        return y


class MLP(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.fc1 = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.fc2 = nn.Linear(4 * config.n_embd, config.n_embd)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class Block(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


class GPTModel(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config

        self.token_embed = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_embed   = nn.Embedding(config.block_size, config.n_embd)
        self.drop = nn.Dropout(config.dropout)

        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size

        pos = torch.arange(0, T, dtype=torch.long, device=idx.device).unsqueeze(0)
        tok_emb = self.token_embed(idx)        # (B, T, C)
        pos_emb = self.pos_embed(pos)          # (1, T, C)
        x = self.drop(tok_emb + pos_emb)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.head(x)  # (B, T, vocab_size)

        loss = None
        if targets is not None:
            B, T, V = logits.shape
            logits_flat = logits.view(B*T, V)
            targets_flat = targets.view(B*T)
            loss = nn.functional.cross_entropy(logits_flat, targets_flat)

        return logits, loss


def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [5]:
import json
import numpy as np

with open("/content/drive/MyDrive/music-project-ml-tokenized/vocab.json", "r") as f:
    vocab = json.load(f)

# char -> int
stoi = vocab["stoi"]

# raw itos may be dict with string keys or a list
raw_itos = vocab["itos"]
if isinstance(raw_itos, dict):
    # keys are strings like "0", "1", ..., convert to int
    itos = {int(k): v for k, v in raw_itos.items()}
else:
    # if it's a list, index is already the id
    itos = {i: ch for i, ch in enumerate(raw_itos)}

vocab_size = len(stoi)
print("Vocab size:", vocab_size)

def encode(text: str) -> np.ndarray:
    """String -> array of token ids."""
    return np.array([stoi[ch] for ch in text if ch in stoi], dtype=np.int64)

def decode(ids) -> str:
    """Iterable of token ids -> string."""
    return "".join(itos[int(i)] for i in ids)


Vocab size: 100


In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

ckpt = torch.load("/content/drive/MyDrive/music-project-ml-results/best_xl_weights.pt",
                  map_location=device)
cfg = ckpt["config"]
print("Loaded config:", cfg)

model = GPTModel(
    GPTConfig(
        vocab_size=vocab_size,
        block_size=256,
        n_layer=cfg["n_layer"],
        n_head=cfg["n_head"],
        n_embd=cfg["n_embd"],
        dropout=0.0,  # no dropout at sampling
    )
).to(device)

model.load_state_dict(ckpt["model_state_dict"])
model.eval()

print("Model loaded for sampling.")


Device: cuda
Loaded config: {'n_layer': 20, 'n_head': 8, 'n_embd': 640}
Model loaded for sampling.


In [7]:
!apt-get install -y abcmidi

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  abcm2ps timidity | pmidi postscript-viewer
The following NEW packages will be installed:
  abcmidi
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 306 kB of archives.
After this operation, 868 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 abcmidi amd64 20220218+ds1-1 [306 kB]
Fetched 306 kB in 1s (209 kB/s)
Selecting previously unselected package abcmidi.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../abcmidi_20220218+ds1-1_amd64.deb ...
Unpacking abcmidi (20220218+ds1-1) ...
Setting up abcmidi (20220218+ds1-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [8]:
import torch.nn.functional as F

@torch.no_grad()
def generate(model, input_ids, max_new_tokens=2000, temperature=1.0, top_k=50):
    model.eval()
    for _ in range(max_new_tokens):
        input_crop = input_ids[:, -model.config.block_size:]
        logits, _ = model(input_crop, None)  # (1, T, vocab)
        logits = logits[:, -1, :] / temperature

        if top_k is not None:
            v, ix = torch.topk(logits, top_k)
            logits_filtered = torch.full_like(logits, -1e10)
            logits_filtered.scatter_(1, ix, v)
            logits = logits_filtered

        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)

        input_ids = torch.cat([input_ids, next_id], dim=1)

    return input_ids[0].tolist()


In [9]:
import os
import re
import subprocess
import random
import torch
import glob

# ================================================================
# 0. PARAMETERS
# ================================================================

MAX_BARS = 64                      # ~1 minute of music
TARGET_MIDI = 10                   # number of MIDI files to collect
MAX_ATTEMPTS = 40                 # stop after this many failures

abc_dir = "/content/drive/MyDrive/music-project-ml-results/unconditional_abc_samples"
midi_dir = "/content/drive/MyDrive/music-project-ml-results/unconditional_midis"

os.makedirs(abc_dir, exist_ok=True)
os.makedirs(midi_dir, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ================================================================
# 1. Cleaning utilities (same as conditional version)
# ================================================================

HEADER_PREFIXES = ("X:", "T:", "M:", "L:", "K:", "Q:", "V:", "C:", "R:")

def is_header_line(line: str) -> bool:
    s = line.lstrip()
    return any(s.startswith(p) for p in HEADER_PREFIXES)

def has_header(lines, prefix: str) -> bool:
    return any(line.lstrip().startswith(prefix) for line in lines)

def has_pitch_chars(line: str) -> bool:
    return any(ch in "ABCDEFGabcdefg" for ch in line)

def clean_abc(text: str) -> str:
    cleaned = []
    for line in text.splitlines():
        s = line.strip()
        if not s:
            continue
        if s.startswith("%"):
            continue
        if is_header_line(s):
            cleaned.append(s)
            continue
        if not has_pitch_chars(s):
            continue
        cleaned.append(s)
    return "\n".join(cleaned)

def add_minimal_header(body: str,
                       index=1,
                       title="Generated Tune",
                       default_meter="4/4",
                       default_key="C"):
    lines = [ln for ln in body.splitlines() if ln.strip()]
    final = []
    if not has_header(lines, "X:"):
        final.append(f"X:{index}")
    if not has_header(lines, "T:"):
        final.append(f"T:{title}")
    if not has_header(lines, "M:"):
        final.append(f"M:{default_meter}")
    if not has_header(lines, "K:"):
        final.append(f"K:{default_key}")
    final.extend(lines)
    return "\n".join(final) + "\n"


# ================================================================
# 2. NEW: truncate ABC after N bars (approx 1 minute)
# ================================================================

def truncate_abc_by_bars(abc_text: str, max_bars: int = 64) -> str:
    lines = abc_text.splitlines()

    header_lines = []
    body_lines = []
    in_header = True

    for ln in lines:
        if in_header and is_header_line(ln):
            header_lines.append(ln)
        else:
            in_header = False
            body_lines.append(ln)

    truncated_body = []
    bar_count = 0

    for ln in body_lines:
        current = []
        for ch in ln:
            if ch == "|":
                bar_count += 1
                if bar_count > max_bars:
                    break
            current.append(ch)
        truncated_body.append("".join(current))
        if bar_count > max_bars:
            break

    return "\n".join(header_lines + truncated_body) + "\n"


# ================================================================
# 3. Use abc2midi to test validity or convert to MIDI
# ================================================================

def abc2midi_ok(abc_path: str, midi_path: str | None = None) -> bool:
    cmd = ["abc2midi", abc_path]
    if midi_path is None:
        cmd += ["-o", os.devnull]
    else:
        cmd += ["-o", midi_path]

    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        errors="ignore",
    )
    return result.returncode == 0


# ================================================================
# 4. UNCONDITIONAL GENERATION LOOP (NO PREFIX)
# ================================================================

success_count = 0
attempt = 0

while success_count < TARGET_MIDI and attempt < MAX_ATTEMPTS:
    attempt += 1
    print(f"\n=== UNCONDITIONAL ATTEMPT {attempt} ===")

    # 1) Generate random seed token
    seed = torch.randint(0, vocab_size, (1, 1), device=device)
    out_ids = generate(model, seed, max_new_tokens=1200, temperature=0.9)
    raw_text = decode(out_ids)

    # 2) Clean ABC
    cleaned = clean_abc(raw_text)

    # 3) Add headers
    final_abc = add_minimal_header(
        cleaned,
        index=attempt,
        title=f"Uncond Sample {attempt}",
        default_meter="4/4",
        default_key="C"
    )

    # 4) TRUNCATE to 1 minute using bar counting
    final_abc = truncate_abc_by_bars(final_abc, max_bars=MAX_BARS)

    # Save ABC
    abc_path = os.path.join(abc_dir, f"uncond_attempt_{attempt}.abc")
    with open(abc_path, "w") as f:
        f.write(final_abc)

    # 5) Convert using abc2midi
    midi_path = os.path.join(midi_dir, f"uncond_success_{success_count+1}.mid")

    if abc2midi_ok(abc_path, midi_path):
        success_count += 1
        print(f"  -> MIDI OK (#{success_count}) saved to {midi_path}")
    else:
        print("  -> abc2midi FAILED (syntax or conversion error)")

print("\n=============== DONE GENERATING UNCONDITIONAL ===============")
print(f"Generated {success_count} MIDI files after {attempt} attempts.")
print(f"ABC stored in:  {abc_dir}")
print(f"MIDI stored in: {midi_dir}")


# ================================================================
# 5. REPORT VALIDITY & CONVERSION RATES
# ================================================================

abc_files = sorted(glob.glob(os.path.join(abc_dir, "*.abc")))
total_abc = len(abc_files)

print(f"\nFound {total_abc} ABC files in: {abc_dir}")

# A) syntactic validity
valid_count = sum(1 for f in abc_files if abc2midi_ok(f, midi_path=None))
valid_pct = 100 * valid_count / total_abc if total_abc else 0

# B) actual MIDI conversion success
midi_files = [x for x in os.listdir(midi_dir) if x.endswith(".mid")]
total_midi = len(midi_files)
midi_pct = 100 * total_midi / total_abc if total_abc else 0

print(f"Syntactically valid ABC: {valid_count}/{total_abc} ({valid_pct:.1f}%)")
print(f"Successful ABC → MIDI:  {total_midi}/{total_abc} ({midi_pct:.1f}%)")

if valid_count > 0:
    midi_over_valid = 100 * total_midi / valid_count
    print(f"MIDI success among valid ABC: {total_midi}/{valid_count} ({midi_over_valid:.1f}%)")


Device: cuda

=== UNCONDITIONAL ATTEMPT 1 ===
  -> MIDI OK (#1) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_1.mid

=== UNCONDITIONAL ATTEMPT 2 ===
  -> MIDI OK (#2) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_2.mid

=== UNCONDITIONAL ATTEMPT 3 ===
  -> MIDI OK (#3) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_3.mid

=== UNCONDITIONAL ATTEMPT 4 ===
  -> MIDI OK (#4) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_4.mid

=== UNCONDITIONAL ATTEMPT 5 ===
  -> MIDI OK (#5) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_5.mid

=== UNCONDITIONAL ATTEMPT 6 ===
  -> MIDI OK (#6) saved to /content/drive/MyDrive/music-project-ml-results/unconditional_midis/uncond_success_6.mid

=== UNCONDITIONAL ATTEMPT 7 ===
  -> MIDI OK (#7) saved to /content/drive/MyDrive/music-proj

In [ ]:
import os
import re
import random
import subprocess
import torch
import glob

# ================================================================
# 1. Three prefixes
# ================================================================

prefix_A = """X: X:1759
T:F\"ur Elise
T:Bagatelle No.25 in A, WoO.59
C:Ludwig van Beethoven
O:Germany
Z:Transcribed by Frank Nordberg - http://www.musicaviva.com
F:http://abc.musicaviva.com/tunes/beethoven-ludwig-van/be059/be059-pno2.abc
V:1 Program 1 0 %Piano
V:2 Program 1 0 bass %Piano
M:3/8
L:1/16
Q:3/8=40
K:Am
V:1
e^d|e^deB=dc|A2 z CEA|B2 z E^GB|c2
"""

prefix_B = """X:670
T:Jingle Bells
C:James Lord Pierpont, 1857
M:2/2
R:Reel
L:1/4
S:Colin Hume's website - chords can also be printed below.
Q:1/2=110
K:G
V:1
%%MIDI chordprog 70
%%MIDI program 57
%%MIDI gchord zzczzzcz
"G"DB AG | "/"D3 D/D/ | "G"DB AG | "C"E4 | "Am"Ec BA | "D"F4 | "D7"dd cA | "G"B4 |
"""

prefix_C = """X: 3
T:Happy Birthday to You
M:3/4
L:1/8
K:G
D>D | E2 D2 G2 | F4 D>D | E2 D2 A2 | G4 D>D |
"""

prefixes = [prefix_A, prefix_B, prefix_C]


# ================================================================
# 2. Cleaning + header utilities (no music21 here)
# ================================================================

HEADER_PREFIXES = ("X:", "T:", "M:", "L:", "K:", "Q:", "V:", "C:", "R:")

def is_header_line(line: str) -> bool:
    s = line.lstrip()
    return any(s.startswith(p) for p in HEADER_PREFIXES)

def has_header(lines, prefix: str) -> bool:
    return any(line.lstrip().startswith(prefix) for line in lines)

def has_pitch_chars(line: str) -> bool:
    # Rough heuristic: must contain at least one pitch letter A–G/a–g
    return any(ch in "ABCDEFGabcdefg" for ch in line)

def clean_abc(text: str) -> str:
    """
    Drop empty lines, comments, and lines with no pitch letters.
    Keep header lines and musically meaningful lines.
    """
    cleaned = []
    for line in text.splitlines():
        s = line.strip()
        if not s:
            continue
        if s.startswith("%"):
            continue
        if is_header_line(s):
            cleaned.append(s)
            continue
        if not has_pitch_chars(s):
            continue
        cleaned.append(s)
    return "\n".join(cleaned)

def add_minimal_header(body: str,
                       index: int = 1,
                       title: str = "Generated Tune",
                       default_meter: str = "4/4",
                       default_key: str = "C") -> str:
    """
    Ensure X:, T:, M:, K: exist once at the top.
    """
    lines = [ln for ln in body.splitlines() if ln.strip()]
    final = []
    if not has_header(lines, "X:"):
        final.append(f"X:{index}")
    if not has_header(lines, "T:"):
        final.append(f"T:{title}")
    if not has_header(lines, "M:"):
        final.append(f"M:{default_meter}")
    if not has_header(lines, "K:"):
        final.append(f"K:{default_key}")
    final.extend(lines)
    return "\n".join(final) + "\n"


# ================================================================
# 2b. NEW: Truncate by number of bars (approx "1 minute" cap)
# ================================================================

def truncate_abc_by_bars(abc_text: str, max_bars: int = 64) -> str:
    """
    Keep headers, then truncate the body after at most `max_bars` barlines ('|').
    This approximates limiting the musical length to around ~1 minute.
    """
    lines = abc_text.splitlines()

    header_lines = []
    body_lines = []
    in_header = True

    for ln in lines:
        # Once we hit the first non-header, treat the rest as body
        if in_header and is_header_line(ln):
            header_lines.append(ln)
        else:
            in_header = False
            body_lines.append(ln)

    bar_count = 0
    truncated_body_lines = []

    for ln in body_lines:
        current_line_chars = []
        for ch in ln:
            if ch == '|':
                bar_count += 1
                if bar_count > max_bars:
                    # Stop including more once we exceed max_bars
                    break
            current_line_chars.append(ch)
        truncated_body_lines.append("".join(current_line_chars))
        if bar_count > max_bars:
            break

    # Reassemble
    full_lines = header_lines + truncated_body_lines
    # Remove trailing totally-empty lines
    full_lines = [l for l in full_lines if l.strip() != ""]
    return "\n".join(full_lines) + "\n"


# ================================================================
# 3. Conditional generator that randomly chooses a prefix
# ================================================================

def generate_with_random_prefix(max_len=1200, temperature=0.9):
    prefix_text = random.choice(prefixes)
    prefix_ids = encode(prefix_text)
    input_ids = torch.tensor(prefix_ids, dtype=torch.long, device=device).unsqueeze(0)
    out_ids = generate(model, input_ids, max_new_tokens=max_len, temperature=temperature)
    return prefix_text, decode(out_ids)


# ================================================================
# 4. Helper: use abc2midi to test syntax or convert
# ================================================================

def abc2midi_ok(abc_path: str, midi_path: str | None = None) -> bool:
    """
    Run abc2midi on abc_path.
    If midi_path is None, send output to /dev/null (syntax check only).
    Returns True iff abc2midi exit code is 0.
    """
    cmd = ["abc2midi", abc_path]
    if midi_path is None:
        cmd += ["-o", os.devnull]
    else:
        cmd += ["-o", midi_path]

    # text=True but ignore decode errors to avoid UnicodeDecodeError
    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        errors="ignore",
    )
    return result.returncode == 0


# ================================================================
# 5. Loop until we get 10 MIDI files (conversion via abc2midi)
#    with truncation to ~1 minute via max_bars
# ================================================================

target_midi = 10
max_attempts = 40
MAX_BARS = 64   # tweak this if you want shorter/longer pieces

abc_dir = "/content/drive/MyDrive/music-project-ml-results/conditional_abc_samples"
midi_dir = "/content/drive/MyDrive/music-project-ml-results/conditional_midis"
os.makedirs(abc_dir, exist_ok=True)
os.makedirs(midi_dir, exist_ok=True)

success_count = 0
attempt = 0

while success_count < target_midi and attempt < max_attempts:
    attempt += 1
    print(f"\n=== CONDITIONAL ATTEMPT {attempt} ===")

    # 1) Generate conditioned on a random prefix
    chosen_prefix, raw = generate_with_random_prefix()

    # 2) Clean & add header
    cleaned = clean_abc(raw)
    final_abc = add_minimal_header(
        cleaned,
        index=attempt,
        title=f"Conditional Sample {attempt}",
        default_meter="4/4",
        default_key="C",
    )

    # 3) Truncate the music to ~1 minute (by bar count)
    final_abc = truncate_abc_by_bars(final_abc, max_bars=MAX_BARS)

    # Save ABC (for inspection)
    abc_path = os.path.join(abc_dir, f"cond_attempt_{attempt}.abc")
    with open(abc_path, "w") as f:
        f.write(final_abc)

    # 4) Convert using abc2midi
    midi_path = os.path.join(midi_dir, f"cond_success_{success_count+1}.mid")
    ok = abc2midi_ok(abc_path, midi_path)
    if ok:
        success_count += 1
        print(f"  -> abc2midi OK (#{success_count}) saved to {midi_path}")
    else:
        print("  -> abc2midi FAILED (syntax or conversion error)")

print("\n================ GENERATION DONE ================")
print(f"Generated {success_count} MIDI files after {attempt} attempts.")
print(f"ABC saved in:  {abc_dir}")
print(f"MIDI saved in: {midi_dir}")


# ================================================================
# 6. Reporting: syntactically valid % and conversion success %
# ================================================================

abc_files = sorted(glob.glob(os.path.join(abc_dir, "*.abc")))
total_abc = len(abc_files)
print(f"\nFound {total_abc} ABC files in {abc_dir}")

# (a) Syntactic validity via abc2midi to /dev/null
valid_count = 0
for path in abc_files:
    if abc2midi_ok(path, midi_path=None):
        valid_count += 1

valid_pct = 100.0 * valid_count / total_abc if total_abc > 0 else 0.0
print(f"Syntactically valid ABC (abc2midi): {valid_count}/{total_abc} ({valid_pct:.1f}%)")

# (b) Conversion success: how many .mid actually produced
midi_files = [f for f in os.listdir(midi_dir) if f.endswith(".mid")]
total_midi = len(midi_files)
midi_pct = 100.0 * total_midi / total_abc if total_abc > 0 else 0.0
print(f"Successful ABC → MIDI (abc2midi): {total_midi}/{total_abc} ({midi_pct:.1f}%)")

# (c) Optional: among syntactically valid ones, how many produced MIDI
midi_over_valid_pct = 100.0 * total_midi / valid_count if valid_count > 0 else 0.0
print(f"MIDI successes among valid ABC: {total_midi}/{valid_count} ({midi_over_valid_pct:.1f}%)")



=== CONDITIONAL ATTEMPT 1 ===
  -> abc2midi OK (#1) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_1.mid

=== CONDITIONAL ATTEMPT 2 ===
  -> abc2midi OK (#2) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_2.mid

=== CONDITIONAL ATTEMPT 3 ===
  -> abc2midi OK (#3) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_3.mid

=== CONDITIONAL ATTEMPT 4 ===
  -> abc2midi OK (#4) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_4.mid

=== CONDITIONAL ATTEMPT 5 ===
  -> abc2midi OK (#5) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_5.mid

=== CONDITIONAL ATTEMPT 6 ===
  -> abc2midi OK (#6) saved to /content/drive/MyDrive/music-project-ml-results/conditional_midis/cond_success_6.mid

=== CONDITIONAL ATTEMPT 7 ===
  -> abc2midi OK (#7) saved to /content/drive/MyDrive/music-project-ml-results/conditio